In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 692567347925603406
xla_global_id: -1
]


In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from config import Config
from utils import normalize
import pickle
import threading

# Definitions

## Paths Declaration
Qui, di seguito, vengono riportati i path relativi al dataset di training, validation e testing. In particolare, il dataset di validation coincide con il primo dataset di testing contenuto nella cartella './LGHG2@n10C_to_25degC/Test'. Infatti, all'interno della directory './LGHG2@n10C_to_25degC/Test' sono presenti 5 possibili insiemi di dati così da poter essere sfruttati durante il testing.

In [7]:
config = Config(
    data_dir='./data',
    train_data_subdir='train/',
    val_data_subdir='test/',
    test_data_subdir='test/'
)

data_dir = ['test', 'train', 'val']
train_data_dir = ['TRAIN_LGHG2@n10degC_to_25degC_Norm_5Inputs.csv']
val_data_dir = ['01_TEST_LGHG2@n10degC_Norm_(05_Inputs).csv', '02_TEST_LGHG2@0degC_Norm_(05_Inputs).csv', '03_TEST_LGHG2@10degC_Norm_(05_Inputs).csv', '04_TEST_LGHG2@25degC_Norm_(05_Inputs).csv']
test_data_dir = ['01_TEST_LGHG2@n10degC_Norm_(05_Inputs).csv', '02_TEST_LGHG2@0degC_Norm_(05_Inputs).csv', '03_TEST_LGHG2@10degC_Norm_(05_Inputs).csv', '04_TEST_LGHG2@25degC_Norm_(05_Inputs).csv']


# Data Importing

## Training Data

In [8]:
train_data_filename = os.listdir(config.get_train_data_dir())[0]
train_data_path = os.path.join(config.get_train_data_dir(), train_data_filename)

train_data = pd.read_csv(train_data_path)
train_data

,V,I,Temp,V_avg,I_avg,SOC
0,0.385148,0.75102,0.303101,0.385148,0.75102,0.206417
1,0.385152,0.75102,0.304591,0.385150,0.75102,0.206417
2,0.385156,0.75102,0.306081,0.385152,0.75102,0.206417
3,0.385160,0.75102,0.307572,0.385154,0.75102,0.206417
4,0.385164,0.75102,0.309062,0.385156,0.75102,0.206417
...,...,...,...,...,...,...
669951,0.478843,0.75102,0.008477,0.459558,0.75102,0.283243
669952,0.478843,0.75102,0.008477,0.459699,0.75102,0.283243
669953,0.478843,0.75102,0.008477,0.459839,0.75102,0.283243
669954,0.478961,0.75102,0.008477,0.459979,0.75102,0.283243


In [9]:
train_data.columns

Index(['V', 'I', 'Temp', 'V_avg', 'I_avg', 'SOC'], dtype='object')

In [10]:
X_train = train_data[['V', 'I', 'Temp', 'V_avg', 'I_avg']].values
X_train.shape

(669956, 5)

In [11]:
X_train

array([[0.38514793, 0.75102009, 0.30310108, 0.38514793, 0.75102009],
       [0.38515183, 0.75102009, 0.30459129, 0.38514988, 0.75102009],
       [0.38515573, 0.75102009, 0.3060815 , 0.38515183, 0.75102009],
       ...,
       [0.47884278, 0.75102009, 0.00847709, 0.45983939, 0.75102009],
       [0.4789612 , 0.75102009, 0.00847709, 0.45997861, 0.75102009],
       [0.4789612 , 0.75102009, 0.00847709, 0.46011672, 0.75102009]])

In [12]:
y_train = train_data['SOC'].values
y_train.shape

(669956,)

In [13]:
y_train

array([0.20641667, 0.20641667, 0.20641667, ..., 0.28324333, 0.28324333,
       0.28324333])

## Validation Data

In [14]:
val_data_filename = os.listdir(config.get_val_data_dir())[3]
val_data_path = os.path.join(config.get_val_data_dir(), val_data_filename)

val_data = pd.read_csv(val_data_path)
val_data

,V,I,Temp,V_avg,I_avg,SOC
0,0.966960,0.748900,0.920678,0.966960,0.748900,1.000000
1,0.966020,0.746992,0.920677,0.966490,0.747946,0.999990
2,0.965901,0.746992,0.917845,0.966294,0.747628,0.999983
3,0.965783,0.747098,0.917845,0.966166,0.747496,0.999973
4,0.965665,0.746992,0.917845,0.966066,0.747395,0.999963
...,...,...,...,...,...,...
47512,0.298614,0.751020,0.926344,0.292723,0.751020,0.136623
47513,0.298614,0.751020,0.926344,0.292761,0.751020,0.136623
47514,0.298614,0.751020,0.926344,0.292798,0.751020,0.136623
47515,0.298614,0.751020,0.929177,0.292834,0.751020,0.136623


In [15]:
X_val = val_data[['V', 'I', 'Temp', 'V_avg', 'I_avg']].values
y_val = val_data['SOC'].values

## Data Normalization

In [16]:
normalized_X_train = normalize(X_train)
normalized_X_train

array([[-0.63365322,  1.        , -1.        , -0.63365322,  1.        ],
       [-0.63908898,  1.        , -1.        , -0.63909772,  1.        ],
       [-0.64456116,  1.        , -1.        , -0.64457869,  1.        ],
       ...,
       [ 0.26690492,  1.        , -1.        ,  0.21572031,  1.        ],
       [ 0.26722387,  1.        , -1.        ,  0.21609528,  1.        ],
       [ 0.26722387,  1.        , -1.        ,  0.21646728,  1.        ]])

In [17]:
normalized_X_val = normalize(X_val)
normalized_X_val

array([[ 1.        , -1.        ,  0.57551079,  1.        , -1.        ],
       [ 0.99571585, -1.        ,  0.58256774,  1.        , -0.99130679],
       [ 0.99642139, -1.        ,  0.55815534,  1.        , -0.99419975],
       ...,
       [-0.98163938,  0.44653221,  1.        , -1.        ,  0.44653221],
       [-0.98183573,  0.44006032,  1.        , -1.        ,  0.44006032],
       [-0.98194943,  0.44002819,  1.        , -1.        ,  0.44002819]])

# Regression

## Linear Regression

In [18]:
from sklearn import linear_model

lregression_model = linear_model.LinearRegression()
lregression_model

LinearRegression()

In [19]:
lregression_model.fit(
    X=normalized_X_train,
    y=y_train
)

LinearRegression()

In [20]:
lregression_model_path = './models/ml/soc_estimation_ml_lregression.sav'
pickle.dump(lregression_model, open(lregression_model_path, 'wb'))

## Gaussian Process Regressor (GPR)

In [21]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

kernel = DotProduct() + WhiteKernel()
gpr_model = GaussianProcessRegressor(kernel=kernel, random_state=0)
gpr_model

GaussianProcessRegressor(kernel=DotProduct(sigma_0=1) + WhiteKernel(noise_level=1),
                         random_state=0)

In [22]:
def training(limited_data, no_batches, model, X_train, y_train):
    print(f'{threading.current_thread().name} is starting a new training session...')
    for i in range(no_batches):
        print(f'{threading.current_thread().name}, session {i+1}/{no_batches}')
        lb = i*limited_data
        ub = (i+1)*limited_data
        gpr_model.fit(
            X=normalized_X_train[lb:ub], 
            y=y_train[lb:ub]
        )

In [23]:
def multithreading_training(no_threads, model, X_train, y_train):
    no_jobs_per_thread = round(len(normalized_X_train) / no_threads)
    limited_data = 500
    no_batches = round(no_jobs_per_thread / limited_data)-1

    threads = []
    for i in range(no_threads):
        t = threading.Thread(
            target=training, 
            args=(
                limited_data, 
                no_batches, 
                gpr_model, 
                normalized_X_train, 
                y_train,
            )
        )
        threads.append(t)
    
    for i in range(no_threads):
        print(f'{threads[i].name} is starting...')
        threads[i].start()
        
    for i in range(no_threads):
        threads[i].join()

In [24]:
no_threads = 8
multithreading_training(
    no_threads=no_threads,
    model=gpr_model, 
    X_train=normalized_X_train, 
    y_train=y_train
)

Thread-11 (training) is starting...
Thread-11 (training) is starting a new training session...
Thread-11 (training), session 1/166
Thread-12 (training) is starting...
Thread-12 (training) is starting a new training session...
Thread-12 (training), session 1/166
Thread-13 (training) is starting...
Thread-13 (training) is starting a new training session...
Thread-13 (training), session 1/166
Thread-14 (training) is starting...
Thread-14 (training) is starting a new training session...
Thread-14 (training), session 1/166
Thread-15 (training) is starting...
Thread-15 (training) is starting a new training session...
Thread-15 (training), session 1/166
Thread-16 (training) is starting...
Thread-16 (training) is starting a new training session...
Thread-16 (training), session 1/166
Thread-17 (training) is starting...
Thread-17 (training) is starting a new training session...
Thread-17 (training), session 1/166
Thread-18 (training) is starting...
Thread-18 (training) is starting a new training

D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The opt

Thread-15 (training), session 3/166


D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Thread-13 (training), session 3/166
Thread-17 (training), session 3/166


...

D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Thread-18 (training), session 165/166
Thread-15 (training), session 166/166


D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Thread-18 (training), session 166/166


D:\virtualenv\myenv\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [25]:
gpr_model_path = './models/ml/soc_estimation_ml_gpr.sav'
pickle.dump(gpr_model, open(gpr_model_path, 'wb'))